In [1]:
import json
import re
import sqlite3
import numpy as np
import pandas as pd
import sqlite3
#import spacy

In [4]:
# Cycle through the json file and grab a certain amount of recipes; limit the number because the file is +100MB
data = []
counter = 0 
with open("20170107-061401-recipeitems.json", encoding = 'utf-8') as json_file:
    # It's neccessary to use readlines(), otherwise the data doesn't come out as lines and can't be read as a json type
    for line in json_file.readlines():
        data.append(json.loads(line))
        counter+= 1
        if counter > 100000:
            break
            

In [17]:
def parse_ingredients(recipe_name, ingredients_list):
    # Use regex to find the patterns for quantity and then the following ingredient stuff; the patterns for quantities look like:
    # ##, #/#, #-#/#; also, remove full stops as this breaks matching and remove everything between () a it's superfluous
    # Parameters: dictionary of recipe_name: [ingredients]
    # Returns a dict of {Name: {Ingredient[n]: "ingredient", measure[n]: measure(s)}
    
    n = 0 
    recipe_ingredients = {}
    
    for item in ingredients_list:
        item = re.sub(" \(.*\)", "", item)
        item = re.sub("[.]", "", item)
        quantity_pattern = re.compile("([0-9][0-9]*/*-*[0-9]*/*[0-9]* \w+)")
        ingredient_quantity = re.findall(quantity_pattern, item)
        # Check if there is a number for the ingredient quanity, if not, work around it
        if len(ingredient_quantity) > 0:
        # The ingredient follows after the quantity, so look there for it
            ingredient_pattern = re.compile(str(ingredient_quantity[-1]+" (.*)"))
            ingredient = ingredient_pattern.search(item)
            # Everything is lower-case for consistency
            recipe_ingredients[str(n)+"_quanitity"] = ' '.join(ingredient_quantity).lower()
            if ingredient is not None:
                recipe_ingredients[str(n)+"_ingredient"] = ingredient[1].lower()
        else:
            recipe_ingredients[str(n)+"_quanitity"] = ' '.join(ingredient_quantity).lower()
            recipe_ingredients[str(n)+"_ingredient"] = item.lower()
        n += 1
    # A very small minority of the items have over 50 ingredients; remove these as this is a data processing error 
    # as these are likely a selection of recipes and including them creates an enormous, sparse table
    if n > 50:
        recipe_ingredients = None
    return recipe_ingredients
        
   
    

In [18]:
# Take the recipe name and ingredients and store those as key:val pairs
recipes = {}
ingredient_dict = {}
for item in data:
    # Ingredients are separated by newline and some have an extra space at the beginning; remove \n and 'of' as well
    ingredients = item['ingredients'].split('\n')
    ingredients = [re.sub(" of "," ", i.strip()) for i in ingredients]
    ingredient_dict[item['name']] = parse_ingredients(item['name'], ingredients)

 
    

In [19]:
recipes_df = np.transpose(pd.DataFrame(ingredient_dict))
#recipes_df.index.name = "Recipe_name"
#recipes_df.set_index(pd.Index(range(len(ingredient_dict.keys()))), inplace=True)
recipes_df.reset_index(inplace = True)
recipes_df.rename(columns={"index":"Recipe_name"}, inplace = True)
recipes_df = recipes_df.replace(np.nan, '', regex=True)


,Recipe_name,0_ingredient,0_quanitity,10_ingredient,10_quanitity,11_ingredient,11_quanitity,12_ingredient,12_quanitity,13_ingredient,...,5_ingredient,5_quanitity,6_ingredient,6_quanitity,7_ingredient,7_quanitity,8_ingredient,8_quanitity,9_ingredient,9_quanitity
0,Drop Biscuits and Sausage Gravy,3 cups,3 cups 3 cups,,,,,,,,...,1 pound,1 pound 1 pound,⅓ cups ⅓ cups,,4 cups,4 cups 4 cups,½ teaspoons ½ teaspoons,,2 teaspoons,2 teaspoons 2 teaspoons
1,Hot Roast Beef Sandwiches,12 whole,12 whole 12 whole,,,,,,,,...,1 tablespoon,1 tablespoon 1 tablespoon,1 tablespoon,1 tablespoon 1 tablespoon,1 tablespoon,1 tablespoon 1 tablespoon,,,,
2,Morrocan Carrot and Chickpea Salad,dressing:,,"fresh mint, torn",1/3 cup 30 g,"for serving: lots toasted almond slices, dried...",,,,,...,"fine sea salt, plus more to taste",1/2 teaspoon,cayenne pepper,1/8 teaspoon,"carrots, shredded on a box grater or sliced wh...",10 ounces,cooked chickpeas,2 cups,"dried pluots, plums, or dates cut into chickp...",2/3 cup 100 g
3,Mixed Berry Shortcake,3 cups,3 cups 3 cups,1 cup,1 cup 1 cup,½ cups ½ cups,,2 tablespoons,2 tablespoons 2 tablespoons,,...,1-¼ cup 1-¼ cup,,½ teaspoons ½ teaspoons,,2 pints,2 pints 2 pints,⅓ cups ⅓ cups,,1 package,1 package 1 package
4,Pomegranate Yogurt Bowl,for each bowl:,,,,,,,,,...,sprinkling toasted sunflower seeds,,optional: whole pomegranate seeds or fresh/dri...,,,,,,,


In [22]:
recipes_df.tail()

,Recipe_name,0_ingredient,0_quanitity,10_ingredient,10_quanitity,11_ingredient,11_quanitity,12_ingredient,12_quanitity,13_ingredient,...,5_ingredient,5_quanitity,6_ingredient,6_quanitity,7_ingredient,7_quanitity,8_ingredient,8_quanitity,9_ingredient,9_quanitity
89344,Crawfish Chowder,butter,1/4 cup,,,,,,,,...,condensed cream mushroom soup,1 can,"whole kernel corn, drained",1 can,"cream cheese, softened",4 ounces,half-and-half cream,2 cups,cayenne pepper,1/2 teaspoon
89345,Crawfish Cakes,egg,1 beaten,,,,,,,,...,cooked and peeled whole crawfish tails,1 pound,"buttery round crackers, crushed",1 sleeve,canola oil,1/4 cup,,,,
89346,Crawfish Bisque,butter,6 tablespoons,,,,,,,,...,salt,1 teaspoon,peeled crawfish tails,1 1 2 pounds,worcestershire sauce,1 teaspoon,"cayenne pepper, or to taste",1 pinch,,
89347,Crawfish Boulettes,cooked and peeled whole crawfish tails,1 package,chopped fresh parsley,2 tablespoons,thinly sliced green onions,3 tablespoons,,,vegetable oil for frying,...,"stale white bread, torn into pieces",5 slices,,1 egg,salt,1 teaspoon,black pepper,1/2 teaspoon,cajun seasoning,2 teaspoons
89348,Crawfish and Corn Soup,butter,1/2 cup,pepper sauce,1 dash,salt to taste,,"crawfish, peeled",1 pound,,...,cream-style corn,2 cans,whole kernel corn,1 can,condensed cream potato soup,1 can,creole seasoning,1/4 teaspoon,worcestershire sauce,1/2 teaspoon


In [20]:
# Import SQLAlchemy and load the recipes into a DB for processing by the bot
from sqlalchemy import create_engine
from sqlalchemy import *
engine = create_engine("sqlite:///C:\\Users\\User\\Documents\\Python Scripts\\Recipes\\recipes.db", echo = False)

In [21]:
recipes_df.to_sql('recipes', con = engine, if_exists='append')

In [75]:
connection = engine.connect()
metadata = MetaData()
test_db = Table('recipes', metadata, autoload = True, autoload_with = engine)

In [76]:
print(test_db.columns.keys())

['index', 'Recipe_name', '0_ingredient', '0_quanitity', '10_ingredient', '10_quanitity', '11_ingredient', '11_quanitity', '12_ingredient', '12_quanitity', '13_ingredient', '13_quanitity', '14_ingredient', '14_quanitity', '15_ingredient', '15_quanitity', '16_ingredient', '16_quanitity', '1_ingredient', '1_quanitity', '2_ingredient', '2_quanitity', '3_ingredient', '3_quanitity', '4_ingredient', '4_quanitity', '5_ingredient', '5_quanitity', '6_ingredient', '6_quanitity', '7_ingredient', '7_quanitity', '8_ingredient', '8_quanitity', '9_ingredient', '9_quanitity']


In [15]:
recipes_df[recipes_df["50_ingredient"] != ""]

,Recipe_name,0_ingredient,0_quanitity,100_ingredient,100_quanitity,101_ingredient,101_quanitity,102_ingredient,102_quanitity,103_quanitity,...,96_ingredient,96_quanitity,97_ingredient,97_quanitity,98_ingredient,98_quanitity,99_ingredient,99_quanitity,9_ingredient,9_quanitity
1715,"Roasted breast of duck with heart stew, gizzar...",ducks,2 small,,,,,,,,...,,,,,,,,,,1 litre
2228,"Pigeon wrapped in filo pastry with Parma ham, ...",pigeons,4 x,,,,,,,,...,,,,,,,,,500ml/18fl oz red wine,
2939,English vegetable minestrone with roasted chum...,100g/3½oz fresh peas,,,,,,,,,...,,,,,,,,,rapeseed oil,1-2 tbsp
3010,Tiffin: a selection of Indian street food,"drumsticks, flesh scored to the bone",12 chicken,,14 litres,white vinegar,2 tbsp,olive oil,2 tbsp,1 onion,...,,1 free,gram flour,2 tbsp,cornflour,1 tbsp,", for frying corn oil",,½ tsp ground ginger,
3012,Slow-roasted lamb shoulder with cashew nut and...,mashed green papaya,2 tbsp,small handful curry leaves,,,3 limes,,3 lemons,,...,butter,2 tbsp,,30 shallots,drizzle clear honey,,", for frying corn oil",,garam masala,1 tsp
3031,Rosemary-smoked pigeon with apricots and almonds,100g/3½oz fresh almonds,,,,,,,,,...,,,,,,,,,,1 garlic
3228,Traditional Wedding Cake,white sugar paste icing,2 lb 4 oz,,,,,,,,...,,,,,,,,,ground cinnamon,2 tsp
3516,"Slow poached chicken, sweetcorn egg, spinach w...",850g/1lb 14oz tinned sweetcorn,,,,,,,,,...,,,,,,,,,fresh thyme,4 sprigs
3792,Venison Wellington with spring peas,300g/10½oz unsalted butter,,,,,,,,,...,,,,,,,,,finely chopped fresh parsley,1 tbsp
3956,Traditional spring wedding cake,white sugar paste icing,4 oz,,,,,,,,...,,,,,,,,,cocoa powder,2 tbsp


In [14]:
recipes_df.iloc[63336,:]

Recipe_name       Cooking For Others: Non-Picnic Brunch
0_ingredient                 frisbee in the park brunch
0_quanitity                                            
100_ingredient                            creme fraiche
100_quanitity                             2 tablespoons
101_ingredient                                         
101_quanitity                                          
102_ingredient                                         
102_quanitity                                          
103_quanitity                                          
104_ingredient                                         
104_quanitity                                          
105_ingredient                                         
105_quanitity                                          
106_ingredient                                         
106_quanitity                                          
107_ingredient                                         
107_quanitity                                   

,0_ingredient,0_quanitity,10_ingredient,10_quanitity,11_ingredient,11_quanitity,12_ingredient,12_quanitity,13_ingredient,13_quanitity,...,5_ingredient,5_quanitity,6_ingredient,6_quanitity,7_ingredient,7_quanitity,8_ingredient,8_quanitity,9_ingredient,9_quanitity
Drop Biscuits and Sausage Gravy,biscuits,,seasoned salt,1/2 teaspoon,"black pepper, more to taste",2 teaspoons,,,,,...,butermilk,1-1/4 cup,sausage gravy,,"breakfast sausage, hot or mild",1 pound,all-purpose flour,1/3 cup,whole milk,4 cups
Hot Roast Beef Sandwiches,dinner rolls or small sandwich buns,12 whole,,,,,,,,,...,poppy seeds,1 tablespoon,spicy mustard,1 tablespoon,horseradish mayo or straight prepared horseradish,1 tablespoon,dash of worcestershire,,"optional dressing ingredients: sriracha, hot s...",
Morrocan Carrot and Chickpea Salad,dressing:,,"fresh mint, torn",1/3 cup 30 g,"for serving: lots toasted almond slices, dried...",,,,,,...,"fine sea salt, plus more to taste",1/2 teaspoon,cayenne pepper,1/8 teaspoon,"carrots, shredded on a box grater or sliced wh...",10 ounces,cooked chickpeas,2 cups,"dried pluots, plums, or dates cut into chickp...",2/3 cup 100 g
Mixed Berry Shortcake,biscuits,,sugar,1/3 cup,orange,1 small,sweet yogurt cream,,plain greek yogurt,1 package,...,"cold butter, cut into pieces",1-1/2 stick,buttermilk,1-1/4 cup,almond extract,1/2 teaspoon,berries,,mixed berries and/or sliced strawberries,2 pints
Pomegranate Yogurt Bowl,for each bowl:,,,,,,,,,,...,sprinkling toasted sunflower seeds,,optional: whole pomegranate seeds or fresh/dri...,,,,,,,


'1 hen 2 chickens'